In [1]:
from functions.Pre_Processing_Casos_Graves import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection._search import GridSearchCV
from sklearn.model_selection._split import StratifiedKFold
from sklearn import svm

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, SelectFdr

import optuna
from sklearn.metrics import f1_score, precision_score, recall_score

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
casos_graves = Pre_Processing_Casos_Graves()

In [3]:
casos_graves.run(columns_symptoms,columns_to_drop,columns_dieases,path)

c:\Users\gabri\Desktop\Git - Tcc\Estudo_dados_de_covid\Codes\functions\Pre_Processing_Casos_Graves.py:282: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[column][i] = 1
c:\Users\gabri\Desktop\Git - Tcc\Estudo_dados_de_covid\Codes\functions\Pre_Processing_Casos_Graves.py:294: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[column][i] = 1


In [4]:
casos_graves.one_hot_encoding()

In [5]:
casos_graves.df

,data_notificacao,sexo,idade,bairro,data_inicio_sintomas,sintomas,outros_sintomas,doencas_preexistentes,outras_doencas_preexistentes,classificacao_final,...,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,age_group_0 - 5,age_group_16 - 25,age_group_26 - 40,age_group_41 - 60,age_group_6 - 15,age_group_61 - 80,age_group_>80
0,2020-05-30,Masculino,53,SAN MARTIN,2020-05-25,"[ALTERAÇÃO/PERDA DE OLFATO E/OU PALADAR, FEBRE...",[],[],[],DESCARTADO,...,0,0,0,0,0,0,1,0,0,0
1,2020-06-02,Feminino,0,ALTO SANTA TEREZINHA,2020-05-30,"[DISPNEIA, FEBRE, SATURAÇÃO O2 < 95, TOSSE]",[],[],[],INCONCLUSIVO,...,0,0,0,1,0,0,0,0,0,0
2,2020-06-02,Feminino,86,ESPINHEIRO,2020-06-01,"[DISPNEIA, SATURAÇÃO O2 < 95]",[],[],[],DESCARTADO,...,0,0,0,0,0,0,0,0,0,1
3,2020-06-01,Masculino,66,CASA FORTE,2020-05-25,[],[],[],[],CONFIRMADO,...,0,0,0,0,0,0,0,0,1,0
4,2020-06-02,Masculino,68,ILHA DO RETIRO,2020-05-30,"[DISPNEIA, SATURAÇÃO O2 < 95, TOSSE]",[],[],[],DESCARTADO,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,2022-06-08,Masculino,77,BREJO DA GUABIRABA,2022-06-06,[DISPNEIA],[],[],[HAS],EM ANÁLISE,...,0,0,0,0,0,0,0,0,1,0
36431,2022-06-09,Masculino,74,NOVA DESCOBERTA,2022-06-06,"[APERTO TORÁCICO, CANSAÇO/FADIGA, DESCONFORTO ...",[],[],[],EM ANÁLISE,...,0,0,0,0,0,0,0,0,1,0
36432,2022-06-10,Masculino,74,BEBERIBE,2022-06-03,"[DISPNEIA, FEBRE, TOSSE]",[],[DIABETES],[],EM ANÁLISE,...,0,0,0,0,0,0,0,0,1,0
36433,2022-06-12,Feminino,73,ENGENHO DO MEIO,2022-06-11,"[APERTO TORÁCICO, DESCONFORTO RESPIRATÓRIO, DI...",[],"[DIABETES, DOENÇAS CARDÍACAS OU VASCULARES]",[AVC],EM ANÁLISE,...,0,0,0,0,0,0,0,0,1,0


In [6]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death','age_group_0 - 5','age_group_16 - 25','age_group_26 - 40','age_group_41 - 60','age_group_6 - 15','age_group_61 - 80','age_group_>80']

df = casos_graves.df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,age_group_0 - 5,age_group_16 - 25,age_group_26 - 40,age_group_41 - 60,age_group_6 - 15,age_group_61 - 80,age_group_>80
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
X = df.drop("death",axis=1)
y = df['death']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
def objective(trial):

        classifier_name = trial.suggest_categorical("classifier", ["RandomForest"])

        bootstrap = trial.suggest_categorical("bootstrap", [True, False])
        #max_depth = trial.suggest_int("max_depth", 10, 100)
        criterion = trial.suggest_categorical("criterion", ['gini','entropy'])
        max_features = trial.suggest_categorical("max_features", ['auto', 'sqrt','log2'])
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        n_estimators = trial.suggest_int("n_estimators", 100, 2000)

        classifier_obj = RandomForestClassifier(
            bootstrap=bootstrap, max_depth=None,
            criterion=criterion, max_features=max_features,
            min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
            n_estimators=n_estimators, random_state=42, n_jobs=-1)

        score = cross_val_score(classifier_obj, X_train, y_train, n_jobs=-1, cv=5,scoring='f1')
        f1 = score.mean()
        return f1

In [11]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-08-16 13:19:58,431] A new study created in memory with name: no-name-15da6f0d-cdb1-41bf-ab86-23264fa7d788
[I 2022-08-16 13:20:33,302] Trial 0 finished with value: 0.37197719187681877 and parameters: {'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 1060}. Best is trial 0 with value: 0.37197719187681877.
[I 2022-08-16 13:20:54,341] Trial 1 finished with value: 0.3467722586509681 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 674}. Best is trial 0 with value: 0.37197719187681877.
[I 2022-08-16 13:21:12,232] Trial 2 finished with value: 0.34640616834971455 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 620}. Best is tria

FrozenTrial(number=32, values=[0.37201616249829644], datetime_start=datetime.datetime(2022, 8, 16, 13, 38, 16, 784234), datetime_complete=datetime.datetime(2022, 8, 16, 13, 38, 53, 996882), params={'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1282}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=32, state=TrialState.COMPLETE, value=None)


In [12]:
best_params={'bootstrap': True, 'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1282}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

Média de fit_time: 21.802356934547426
Média de score_time: 0.8925668239593506
Média de test_f1: 0.36145156145577506
Média de test_precision: 0.537176482389967
Média de test_recall: 0.2729895625279607


In [13]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

Precisão: 0.5281437125748503
Revocação: 0.24378109452736318
F1-score: 0.33358547655068077
